In [1]:
import os

import numpy as np
import pandas as pd
import haven.db as db

os.environ['AWS_PROFILE'] = 'admin'
os.environ['HAVEN_DATABASE'] = 'haven'

In [ ]:
features = {
    "normed_log_mlt": (-1.0, 1.5, 0.15),
    "normed_log_npp": (-4.0, 3.0, 0.4), 
    "normed_distance": (0.0, 1.0, 0.05),
    "water_heading": (-np.pi, np.pi, 0.3),
    "movement_heading": (-np.pi, np.pi, 0.3)
}
dataframes = {
    col: pd.DataFrame({col: np.arange(lower, upper + step, step)})
    for col, (lower, upper, step) in features.items()
}
product = 1.0
for col, dataframe in dataframes.items():
    print(col, dataframe.shape[0])
    product *= dataframe.shape[0]
print(
    "Final Size Will Be:", product
)

In [ ]:
cols = list(dataframes.keys())
result = dataframes[cols[0]]
for col in cols[1:]:
    result = result.merge(
        dataframes[col], how='cross'
    )
result = result.reset_index(drop=True).reset_index().rename(columns={'index': '_individual'})
result['_decision'] = result['_individual']
result['_choice'] = result['_individual']
result['_train'] = False
result

In [4]:
db.write_data(
    result, 'movement_model_hypercube_v3', ['_train']
)